# For the following exercises, you'll need to load several datasets using the pydataset library. (If you get an error when trying to run the import below, use pip to install the pydataset package.)

- from pydataset import data

When the instructions say to load a dataset, you can pass the name of the dataset as a string to the data function to load the dataset. You can also view the documentation for the data set by passing the show_doc keyword argument.

mpg = data('mpg') 
- #load the dataset and store it in a variable
- #data('mpg', show_doc=True) 
- #view the documentation for the dataset

In [1]:
from pydataset import data
import pandas as pd
import numpy as np

# 1. Load the mpg dataset. Read the documentation for it, and use the data to answer these questions:

In [ ]:
#creates a variable holding the mpg dataframe
mpg_df = data('mpg')
#to read the documentation of mpg
data('mpg', show_doc = True)

### On average, which manufacturer has the best miles per gallon?

In [ ]:
#creates a new column of the average of the city and highway mpg
mpg_df['overall_mpg'] = (mpg_df.cty + mpg_df.hwy) / 2
#creates a new df for the values grouped by the manufacturer
#displaying 1 shows us the highest overall mpg
average = mpg_df.groupby('manufacturer').mean()
average = average.sort_values(by='overall_mpg', ascending = False)
average.head(1)
#best_mpg = average.head(1)
#best_mpg

### How many different manufacturers are there?

In [ ]:
#value counts finds the count of each unique value
#using count at the end, sums the unique manufactures
count = mpg_df.manufacturer.value_counts().count()
print('There are',count,'different maufactures in mpg.')

### How many different models are there?

In [ ]:
#value counts finds the count of each unique value
#using count at the end, sums the unique models
count = mpg_df.model.value_counts().count()
print('There are',count,'different models in mpg.')

### Do automatic or manual cars have better miles per gallon?

In [ ]:
#creates a new df of only the automatic cars
auto = mpg_df[mpg_df.trans.str.contains('auto')]

#creates a new data frame of only the manual cars
manual = mpg_df[mpg_df.trans.str.contains('manual')]

#displays the average overall mpg of all automatic cars
print('The overall mpg of automatic cars is: ',auto.overall_mpg.mean())

#displays the average overall mpg of all manual cars
print('The overall mpg of manual cars is: ',manual.overall_mpg.mean())

#prints the higher mpg hardcoded
print('Manual cars have better overall mileage than automatic cars.')

# 2. Joining and Merging

### Copy the users and roles dataframes from the examples above.

In [ ]:
#creates users data frame
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})

#creates roles data frame
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})

### A left join

In [ ]:
#displaying a left join of users and roles
pd.merge(users, roles, left_on='role_id', right_on='id', how='left')

### What do you think a right join would look like?

We are considering the roles table as the right join. For a right join, it takes everything from the right table, and everything shared between the two tables. What is not shared in the left table with the right table is not included. So the values that are not defined in the left table are shown as NaN.
"Returns all records from the right table (table2), and the matched records from the left table (table1). The result is NULL from the left side, when there is no match."
https://www.w3schools.com/sql/sql_join_right.asp

In [ ]:
#a right join of users and roles
pd.merge(users, roles, left_on='role_id', right_on='id', how='right')

### An outer join? 

In an outer join, unmatched rows in one or both tables can be returned.

In [ ]:
#an outer join of users and roles
pd.merge(users, roles, left_on='role_id', right_on='id', how='outer')

### What happens if you drop the foreign keys from the dataframes and try to merge them?

In [ ]:
#creating the tables without the shared key
#a join can not happen without these
#creates users data frame
users2 = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
})

#creates roles data frame
roles2 = pd.DataFrame({
    'name': ['admin', 'author', 'reviewer', 'commenter']
})

pd.merge(users2, roles2, left_on='role_id', right_on='id', how='left')

# 3. Getting data from SQL databases

### Create a function named get_db_url. It should accept a username, hostname, password, and database name and return a url formatted like in the examples in this lesson.

In [2]:
def get_db_url(db_name):
    from env import host, username, password
    url = f'mysql+pymysql://{username}:{password}@{host}/{db_name}'
    return (url)

### Use your function to obtain a connection to the employees database.

In [8]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', get_db_url('employees'))

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


### Once you have successfully run a query:

### Intentionally make a typo in the database url. What kind of error message do you see?

In [ ]:
get_db_url_typo = f'mysql+pymysql://{username}123:{password}@{host}/employees'

pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', get_db_url_typo)

Error message was...
OperationalError: (1045, "Access denied for user...

### Intentionally make an error in your SQL query. What does the error message look like?

In [ ]:
pd.read_sql('SELECT * FRO employees LIMIT 5 OFFSET 50', get_db_url)

- when table spelled wrong: ProgrammingError: (1146, "Table 'employees.employee' doesn't exist")
- when syntax spelled wrong: ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'FRO employees LIMIT 5 OFFSET 50' at line 1")

### Read the employees and titles tables into two separate dataframes

In [3]:
employees = pd.read_sql('SELECT * FROM employees', get_db_url('employees'))
titles = pd.read_sql('SELECT * FROM titles', get_db_url('employees'))

### Visualize the number of employees with each title.

In [4]:
#using historical data, we did not filter out the old title dates from above
#we joined the tables with a left join using the emp_no
emp_with_titles = pd.merge(employees, titles, left_on='emp_no', right_on='emp_no', how='left')
#value counts shows us the counts of the distinct title names
emp_with_titles.title.value_counts()

Engineer              115003
Staff                 107391
Senior Engineer        97750
Senior Staff           92853
Technique Leader       15159
Assistant Engineer     15128
Manager                   24
Name: title, dtype: int64

In [5]:
#this time we create a titles data table with only the current employee titles
titles_current = pd.read_sql('SELECT * FROM titles WHERE to_date > CURDATE()', get_db_url('employees'))
#we join the two together with the emp_no
emp_with_titles_current = pd.merge(employees, titles_current, left_on='emp_no', right_on='emp_no', how='left')
#value counts shows us the counts of the distinct title names
emp_with_titles_current.title.value_counts()

Senior Engineer       85939
Senior Staff          82024
Engineer              30983
Staff                 25526
Technique Leader      12055
Assistant Engineer     3588
Manager                   9
Name: title, dtype: int64

### Join the employees and titles dataframes together. Visualize how frequently employees change titles.

In [21]:
#we joined the two tables together as emp_with_titles
changes = emp_with_titles.emp_no.value_counts().sum()
emp_total = emp_with_titles.emp_no.value_counts().count()
title_rate = changes / emp_total
print('Employees change titles at a rate of', title_rate, 'per employee.')

Employees change titles at a rate of 1.4775751273231474 per employee.


### For each title, find the hire date of the employee that was hired most recently with that title.

In [56]:
emp_with_titles.groupby('title')['hire_date'].max()

title
Assistant Engineer    1999-12-24
Engineer              2000-01-28
Manager               1992-02-05
Senior Engineer       2000-01-01
Senior Staff          2000-01-13
Staff                 2000-01-12
Technique Leader      1999-12-31
Name: hire_date, dtype: object

### Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL and python/pandas code)

# 4. Use your get_db_url function to help you explore the data from the chipotle database. Use the data to answer the following questions:

In [14]:
orders = pd.read_sql('''SELECT * FROM orders''',get_db_url('chipotle'))
orders.head()

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98


### What is the total price for each order?

In [15]:
orders['float_price'] = orders.item_price.str.replace('$','').astype(float)

In [8]:
price_per_order = pd.DataFrame(orders.groupby('order_id')['float_price'].sum())

In [12]:
price_per_order['str_price'] = orders.item_price.str.replace('$','').astype(str)

In [16]:
price_per_order

,float_price,str_price
order_id,,
1,11.56,3.39
2,16.98,3.39
3,12.67,2.39
4,21.00,16.98
5,13.70,10.98
...,...,...
1830,23.00,2.15
1831,12.90,1.25
1832,13.20,21.96


### What are the most popular 3 items?

In [51]:
orders.item_name.value_counts().head(3)

Chicken Bowl           726
Chicken Burrito        553
Chips and Guacamole    479
Name: item_name, dtype: int64

### Which item has produced the most revenue?